### Import of data

In [115]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

### Data we going to use in csv

In [116]:
features = ["hktv_ad_id", "order_value", "primary_category_name_en"]

### Import csv, from 2021 November to 2022 November, each months 1st day

In [117]:
csv_1 = pd.read_csv('csv/Public_Transaction_2021-11-01.csv', usecols=features)
csv_2 = pd.read_csv('csv/Public_Transaction_2021-12-01.csv', usecols=features)
csv_3 = pd.read_csv('csv/Public_Transaction_2022-01-01.csv', usecols=features)
csv_4 = pd.read_csv('csv/Public_Transaction_2022-02-01.csv', usecols=features)
csv_5 = pd.read_csv('csv/Public_Transaction_2022-03-01.csv', usecols=features)
csv_6 = pd.read_csv('csv/Public_Transaction_2022-04-01.csv', usecols=features)
csv_7 = pd.read_csv('csv/Public_Transaction_2022-05-01.csv', usecols=features)
csv_8 = pd.read_csv('csv/Public_Transaction_2022-06-01.csv', usecols=features)
csv_9 = pd.read_csv('csv/Public_Transaction_2022-07-01.csv', usecols=features)
csv_10 = pd.read_csv('csv/Public_Transaction_2022-08-01.csv', usecols=features)
csv_11 = pd.read_csv('csv/Public_Transaction_2022-09-01.csv', usecols=features)
csv_12 = pd.read_csv('csv/Public_Transaction_2022-10-01.csv', usecols=features)
csv_13 = pd.read_csv('csv/Public_Transaction_2022-11-01.csv', usecols=features)

### Joining all the csv into a large dataframe

In [118]:
df_1 = pd.concat([csv_1,csv_2,csv_3,csv_4,csv_5,csv_6,csv_7,csv_8,csv_9,csv_10,csv_11,csv_12,csv_13], ignore_index=True)

### Number of Data

In [119]:
df_1.shape[0]

3030730

### Read the user's data including gender and age range

In [120]:
f = open("sampleJSON.txt", encoding='utf-8').read()
feature = ["hktv_ad_id", "gender", "age_range"]
t = f.split("\n")
t.pop()
empty = []
for i in t:
    rows = json.loads(i)
    empty.append([rows["hktv_ad_id"], rows["gender"], rows["age_range"]])
df_2 = pd.DataFrame(empty, columns=feature)

### order value : From range (i.e. 5000-6000) to value (5000+6000 /2 = 5500)

In [121]:
#Make grouped data to numerical data

data = []
for i in range(len(df_1['order_value'])):
    num = df_1['order_value'][i].split(' - ')
    data.append((int(num[0])+int(num[1]))/2)

df_1["price"] = data

### same as order value, from age_range ot age

In [122]:
#Make grouped data to numerical data

data = []
for i in range(len(df_2['age_range'])):
    if df_2['age_range'][i] == None or df_2['age_range'][i] == "":
        data.append(np.nan)
        continue
    if df_2['age_range'][i].split('_')[1] == '55plus':
        data.append(60)
    elif df_2['age_range'][i].split('_')[1] == "12":
        data.append(10)
    else:
        num = df_2['age_range'][i].split('_')
        data.append((int(num[1])+int(num[2]))/2)

df_2['age'] = data

### Merging the transaction table and user data table, by the customer id (hktv_ad_id)

In [123]:
df = pd.merge(df_1, df_2, on="hktv_ad_id")
df.drop(labels=["order_value", "age_range"], axis=1, inplace=True)

In [124]:
#dealing with null and none data?

#price -> mean according to?
df['price'].fillna(value = df['price'].mean(), inplace = True)
#age -> mean according to?
df['age'].fillna(value = df['age'].mean(), inplace = True)
#gender = none
df.dropna(inplace=True)

In [125]:
#Check for the missing value again
df.isna().sum()

hktv_ad_id                  0
primary_category_name_en    0
price                       0
gender                      0
age                         0
dtype: int64

In [126]:
#Rename the 'primary_category_name_en' to 'category'
df.rename(columns={'primary_category_name_en':'category', 'hktv_ad_id':'user'}, inplace=True)
#Reorder to let the target value to be the last column
df = df.reindex(columns=["user", "gender", "age", "category", "price"])

In [127]:
from sklearn.preprocessing import LabelEncoder

In [128]:
l1 = LabelEncoder()
cat = ['category', 'gender']
for parameter in cat:
    l1.fit(df[parameter])
    df[parameter] = l1.transform(df[parameter])

In [129]:
# Number of data after merge and concating all csv
df.shape[0]

106446

In [130]:
df.head()

,user,gender,age,category,price
8,LHXRgMIvLsJIsBZ9EcE6h8jLWkWboqseR1ftfkTPD/U=,0,22.5,12,324.5
17,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
18,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
19,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5
20,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5


# Dimention Reduction JT <hr>

In [131]:
df.head()

,user,gender,age,category,price
8,LHXRgMIvLsJIsBZ9EcE6h8jLWkWboqseR1ftfkTPD/U=,0,22.5,12,324.5
17,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
18,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
19,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5
20,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5


In [132]:
from sklearn.preprocessing import StandardScaler

In [133]:
variables = ['category', 'gender', 'age']
x = df.loc[:, variables].values
y = df.loc[:,['price']].values
x = StandardScaler().fit_transform(x)
x = pd.DataFrame(x)

In [134]:
from sklearn.decomposition import PCA

In [135]:
pca = PCA()
x_pca = pca.fit_transform(x)
x_pca = pd.DataFrame(x_pca)
x_pca.head()

,0,1,2
0,-1.550585,-0.286133,-0.960722
1,-0.851856,-0.547489,-0.397440
2,-0.851856,-0.547489,-0.397440
3,-0.851856,-0.547489,-0.397440
4,-0.851856,-0.547489,-0.397440


### From Below, we can see that only two parameter cannot even capture 70% of the variance of the data, therefore we won't conduct any dimention reduction here.

In [136]:
explained_variance = pca.explained_variance_ratio_
explained_variance[:2].sum()

0.6851997741887046

In [137]:
x_pca['price']=y
x_pca.columns = ['category', 'gender', 'age', 'price']
x_pca.head()

,category,gender,age,price
0,-1.550585,-0.286133,-0.960722,324.5
1,-0.851856,-0.547489,-0.397440,424.5
2,-0.851856,-0.547489,-0.397440,424.5
3,-0.851856,-0.547489,-0.397440,24.5
4,-0.851856,-0.547489,-0.397440,24.5


# Regression Nathalie<hr>

## Thing's to do <br>
<h3><ol>
<li> train test split: Use sklearn library, 80% training 20% test</li>
<li> Model Training: use the best degree aka the best fit line</li>
<li> Performance Testing: Use R2 and RMS From HW 5</li>
</ol></h3>

In [138]:
df.head()

,user,gender,age,category,price
8,LHXRgMIvLsJIsBZ9EcE6h8jLWkWboqseR1ftfkTPD/U=,0,22.5,12,324.5
17,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
18,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,424.5
19,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5
20,W5UIVAX8xv2iZFRYSjDEOmX3wcpAEQvU+SW8p7hAFO4=,0,30.0,12,24.5


In [139]:
print(len(df))

106446


For 3 predictors (), the equation includes all the possible combinations of different order polynomials. This is known as Multi-dimensional Polynomial Regression.

1. Import all libraries needed

In [140]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import math
import seaborn as sns

2. Split the dataframe into traing set and test set

In [141]:
training_size = 80
test_size = 20

# setting the values for independent variable x and dependent variable y
features = ['gender','age','category']
#x = df.loc[:, features]
#y = df.loc[:, ['price']]

X = df[['gender', 'age', 'category']]
Y = df['price']

df = df.sample(frac=1).reset_index(drop=True)# shuffle the data

x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, train_size = .8)
# random_state is a pseudo-random number parameter that allows you to reproduce the same train test split each time you run the code. 

In [142]:
print(X)

        gender   age  category
8            0  22.5        12
17           0  30.0        12
18           0  30.0        12
19           0  30.0        12
20           0  30.0        12
...        ...   ...       ...
175569       1  30.0        13
175570       1  30.0        13
175571       0  30.0        13
175572       0  30.0        13
175573       0  30.0        13

[106446 rows x 3 columns]


In [143]:
type(X)
type(Y)

pandas.core.series.Series

In [144]:
x_train

,gender,age,category
130037,0,50.000000,12
108356,1,30.000000,12
13771,0,30.000000,12
84125,0,35.822573,12
84894,1,60.000000,8
...,...,...,...
33357,1,50.000000,12
74718,0,50.000000,12
69808,0,35.822573,12
71366,1,35.822573,12


In [145]:
y_train

130037     274.5
108356     524.5
13771      124.5
84125      424.5
84894      574.5
           ...  
33357      124.5
74718      424.5
69808      624.5
71366     1249.5
111662     624.5
Name: price, Length: 85156, dtype: float64

In [146]:
df.dtypes

user         object
gender        int64
age         float64
category      int64
price       float64
dtype: object

3. Build the Regression Model
   
    3.1. Multiple linear Regression

In [147]:
mlr = LinearRegression()
mlr.fit(x_train, y_train)

LinearRegression()

In [148]:
# model equation of multiple linear regression
print("Intercept: " ,mlr.intercept_)
print("Coefficients:")
list(zip(x, mlr.coef_))

Intercept:  999.8047215539011
Coefficients:


[(0, -20.612674718910032), (1, -2.631137351991721), (2, -12.965201069387424)]

In [149]:
# prediction of test set
y_pred_mlr= mlr.predict(x_test)
print("Prediction for test set: {}".format(y_pred_mlr))

Prediction for test set: [740.29238332 740.29238332 853.68980734 ... 712.66544112 795.77437496
 729.35552407]


In [150]:
# Since it is difficult to plot out a graph with 4 parameter, so I will only plot out the graph comparsion between the actual value and predicted value
# Actual values and the predicted values
mlr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_pred_mlr})
mlr_diff.head()

,Actual value,Predicted value
130357,574.5,740.292383
75652,1149.5,740.292383
80962,374.5,853.689807
159309,424.5,740.292383
172962,374.5,804.183791


 Evaluation of regression model

In [151]:
score_mlr = mlr.score(x_test, y_test)
print(score_mlr)

0.0013665942044338575


 3. Build the Regression Model
 
    3.2 LASSO Regression Model

    LASSO Regression comes in handy when there is too many features, some of them are can be completely eliminated by setting the coefficients to zero, known as "L1-norm 
    Regularization".

In [152]:
from sklearn.linear_model import Lasso

In [153]:
# define model
# create an instance of Lasso Regression implementation
model = Lasso(alpha=1.0)
# fit the Lasso model
model.fit(x_train, y_train)
# make prediction
y_pred_lasso = model.predict(x_test)

 Evaluation of regression model

In [154]:
model.score(x_test,y_test)

0.0013869653257722048

3. Build the Regression Model

    3.3 Random Forest

# KNN Tree

In [155]:
from sklearn.neighbors import KNeighborsRegressor

In [156]:
knn_model = KNeighborsRegressor(n_neighbors=3)

In [157]:
knn_model.fit(x_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [158]:
from sklearn.metrics import mean_squared_error
from math import sqrt
train_preds = knn_model.predict(x_train)
mse = mean_squared_error(y_train, train_preds)
print(mse)

1290167.4092580413


# Support Vector Machine

In [168]:
df['age'] = df['age'].astype('int')
df

,user,gender,age,category,price
0,eLG1x1kmfHI6Y6yD9V8Jnmj3T4pA8LNxRxiA5GgvPCc=,0,35,12,524.5
1,CDer6pZRybucggIPO/hCTHhOayCySvvOhfORipqIYyk=,0,35,8,474.5
2,d/2ctS3CC6FTe3paNFG5tC5wdULFfGgVpCkGaItCtf4=,1,35,12,374.5
3,NsUhtJeAv+s7wc05iSQu4r66INW2lh/qpM6DC7EG/2U=,0,60,12,374.5
4,dKvZ4zpkcaL/IW/ulkTnbvDtdYtsp0+JITkqWJ192/M=,0,30,12,674.5
...,...,...,...,...,...
106441,tZzkBF321V7HUFGckJxSSXSx2JBUr2uDS6iQKe7vKfU=,1,50,12,224.5
106442,ZUY/oUj+HrlLU65XrgHL+n5XVjNPHNnM6vBa8QvZ084=,1,60,12,1249.5
106443,ip3Yaqu6eaeMWreDwmZe0WZTAkXYBmufLeAHf8pQwQc=,0,60,12,374.5
106444,PK6dzrWgP3m1y/aRoL9o5vBsr1eRwXExD5aVoLxegB0=,0,35,12,874.5


,user,gender,age,category,price
0,eLG1x1kmfHI6Y6yD9V8Jnmj3T4pA8LNxRxiA5GgvPCc=,0,35,12,524.5
1,CDer6pZRybucggIPO/hCTHhOayCySvvOhfORipqIYyk=,0,35,8,474.5
2,d/2ctS3CC6FTe3paNFG5tC5wdULFfGgVpCkGaItCtf4=,1,35,12,374.5
3,NsUhtJeAv+s7wc05iSQu4r66INW2lh/qpM6DC7EG/2U=,0,60,12,374.5
4,dKvZ4zpkcaL/IW/ulkTnbvDtdYtsp0+JITkqWJ192/M=,0,30,12,674.5
...,...,...,...,...,...
106441,tZzkBF321V7HUFGckJxSSXSx2JBUr2uDS6iQKe7vKfU=,1,50,12,224.5
106442,ZUY/oUj+HrlLU65XrgHL+n5XVjNPHNnM6vBa8QvZ084=,1,60,12,1249.5
106443,ip3Yaqu6eaeMWreDwmZe0WZTAkXYBmufLeAHf8pQwQc=,0,60,12,374.5
106444,PK6dzrWgP3m1y/aRoL9o5vBsr1eRwXExD5aVoLxegB0=,0,35,12,874.5


In [165]:
x_train

,gender,age,category
130037,0,50.000000,12
108356,1,30.000000,12
13771,0,30.000000,12
84125,0,35.822573,12
84894,1,60.000000,8
...,...,...,...
33357,1,50.000000,12
74718,0,50.000000,12
69808,0,35.822573,12
71366,1,35.822573,12


In [162]:
from sklearn.svm import SVC
clf = SVC(kernel = "linear")
clf.fit(x_train, y_train)
y_pred_svm = clf.predict(x_test)

ValueError: Unknown label type: 'continuous'

# Decision Tree   Joseph<hr>

In [ ]:
df.head()

,user,gender,age,category,price
0,qPPZIuTeK9+OUznniwPejjvZ73l3erAdXTcZ1fOGOQo=,0,35.822573,8,374.5
1,kB61mwAQ9hreWIyYjnd/KSKuAYO1gy8fYXEMZuC8ye8=,0,30.000000,12,374.5
2,l1a3JTB/2mBspLk3nB/kZiWgq/drd51QaiL3ntppcDA=,0,50.000000,12,424.5
3,3fpJ3jJllfQXbQTDoJjbgmYWrZk3DtkOE6HBMW/zUFw=,1,50.000000,8,1049.5
4,lAJh4FLIOtNu2eBGJTmdzRHwC6/2bznmvUMYAuy/I5k=,0,35.822573,12,374.5


## Thing's to do <br>
<h3><ol>
<li> train test split: Use sklearn library, 80% training 20% test</li>
<li> Model Training</li>
<li> Performance Testing: Use R2 and RMS From HW 5</li>
</ol></h3>

# Visuallization (HW4) <hr>

## Thing's to do <br>
<h3><ol>
<li>Visualize the dimention reduction data</li>
<li>Visualize the regression line with data point</li>
</ol></h3>